In [3]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector
from scipy.optimize import minimize
from functools import partial
from math import ceil, log2
from PIL import Image
import matplotlib.pyplot as plt

# Step 1: Normalize amplitudes for amplitude encoding
def normalize_amplitudes(amplitudes):
    norm = np.linalg.norm(amplitudes)
    if norm == 0:
        raise ValueError("Amplitudes norm is zero. Cannot normalize.")
    return amplitudes / norm

# Step 2: Preprocess galaxy images
def preprocess_image(image_path):
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = image.resize((32, 32))  # Resize to ensure consistent input size
    image_array = np.array(image).flatten()  # Flatten into a 1D vector
    return normalize_amplitudes(image_array)

# Step 3: Amplitude encoding function
def amplitude_encoding(amplitudes):
    n_qubits = ceil(log2(len(amplitudes)))
    padded_length = 2 ** n_qubits
    if len(amplitudes) < padded_length:
        amplitudes = np.pad(amplitudes, (0, padded_length - len(amplitudes)))
    amplitudes = normalize_amplitudes(amplitudes)
    qc = QuantumCircuit(n_qubits)
    qc.initialize(amplitudes, range(n_qubits))
    return qc

# Step 4: Define the parameterized quantum circuit
def u_theta(theta, n_qubits):
    twolocal = TwoLocal(
        n_qubits,
        rotation_blocks=["ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=2,
    )
    num_params = len(twolocal.parameters)
    if len(theta) != num_params:
        raise ValueError(f"Expected {num_params} parameters but got {len(theta)}.")
    param_mapping = {param: value for param, value in zip(twolocal.parameters, theta)}
    twolocal = twolocal.assign_parameters(param_mapping)
    qc = QuantumCircuit(n_qubits)
    qc.compose(twolocal, inplace=True)
    return qc

# Step 5: Multi-Class Galaxy Classification
def classify_galaxy(feature, theta, n_qubits, num_classes):
    qc = amplitude_encoding(feature)
    unitary_circuit = u_theta(theta, n_qubits)
    qc.compose(unitary_circuit, inplace=True)

    statevector = Statevector.from_instruction(qc)
    probabilities = np.abs(statevector.data) ** 2  # Probability distribution

    # Assign each galaxy type to a portion of the quantum state space
    bin_size = len(probabilities) // num_classes
    class_probs = [np.sum(probabilities[i * bin_size : (i + 1) * bin_size]) for i in range(num_classes)]
    
    return np.argmax(class_probs)  # Return class with highest probability

# Step 6: Define the cost function
def cost_function(theta, features, labels, n_qubits, num_classes):
    total_error = 0
    for feature, label in zip(features, labels):
        predicted_label = classify_galaxy(feature, theta, n_qubits, num_classes)
        total_error += (label - predicted_label) ** 2  # Squared error
    return total_error / len(features)

# Step 7: Train the quantum classifier
def train_classifier(features, labels, n_qubits, num_classes):
    initial_theta = np.random.uniform(0, 2 * np.pi, 18)  # Random parameters

    result = minimize(
        fun=partial(cost_function, features=features, labels=labels, n_qubits=n_qubits, num_classes=num_classes),
        x0=initial_theta,
        method="COBYLA",
        options={"maxiter": 1000, "disp": True},
    )
    return result.x

# Step 8: Main Execution
if __name__ == "__main__":
    n_qubits = 6
    num_classes = 2  # Spiral, Elliptical

    # Load galaxy images and labels
    feature_1 = preprocess_image(r'C:\Users\User\Downloads\pixel.png')  
    feature_2 = preprocess_image(r'C:\Users\User\Downloads\pixel (2).png')
    features = [feature_1, feature_2]
    labels = [0, 1]  # Spiral = 0, Elliptical = 1

    # Train classifier
    optimal_theta = train_classifier(features, labels, n_qubits, num_classes)

    # Test on a new galaxy image
    test_feature = preprocess_image(r'C:\Users\User\Downloads\pixel (2).png')
    predicted_label = classify_galaxy(test_feature, optimal_theta, n_qubits, num_classes)

    galaxy_types = ["Spiral", "Elliptical"]
    print(f"Predicted Galaxy Type: {galaxy_types[predicted_label]}")

Predicted Galaxy Type: Elliptical
